# Exploratory Data Analysis for Fees


## Conclusions and key findings from EDA:

Fees:


- Missing data:
    - 4 entries seem to have cash_request_id missing. Assuming that this data can not be recovered, they will need to be cleaned.
    - At first glance, it seems all the rest is correct:
        - `total_amount` seems to be 5.0 for almost all fees
            - Out of 21k entries, there's only 1 fee where total_amount is 10. All the others have total_amount = 5
            - We'll assume that it's not a mistake and keep it as it is.
        - `paid_at` is missing for 5530 entries but at first glance it may be normal (e.g. if the fee is still pending payment)
        - Some other columns also have missing data ("category", "from_date", "to_date") but at first glance it seems normal (those columns only apply to specific cases)

- Format:
    - All columns with dates ("created_at", "updated_at", "paid_at", "from_date", "to_date") need to be converted to a valid date format. 
        - Note: at the moment they're stored as str and float (those with float are likely because the data is NaN)



<br>
<br>


In [1]:
import pandas as pd

fees = pd.read_csv("project_dataset\\extract - fees - data analyst - .csv")


## EDA: fees


In [34]:
display(fees.shape)
display(fees.columns)
# display(fees.head())
# display(fees.sample(10))
# display(fees.tail())

(21061, 13)

Index(['id', 'cash_request_id', 'type', 'status', 'category', 'total_amount',
       'reason', 'created_at', 'updated_at', 'paid_at', 'from_date', 'to_date',
       'charge_moment'],
      dtype='object')

In [9]:
display(fees.describe())

,id,cash_request_id,total_amount
count,21061.000000,21057.000000,21061.000000
mean,10645.355111,16318.449162,5.000237
std,6099.315256,6656.149949,0.034453
min,1.000000,1456.000000,5.000000
25%,5385.000000,11745.000000,5.000000
50%,10652.000000,17160.000000,5.000000
75%,15925.000000,21796.000000,5.000000
max,21193.000000,27010.000000,10.000000


In [11]:
# Get fees without cash_request_id

fees[fees["cash_request_id"].isna()]


,id,cash_request_id,type,status,category,total_amount,reason,created_at,updated_at,paid_at,from_date,to_date,charge_moment
1911,2990,NaN,instant_payment,cancelled,NaN,5.0,Instant Payment Cash Request 11164,2020-08-06 22:42:34.525373+00,2020-11-04 16:01:17.296048+00,NaN,NaN,NaN,after
1960,3124,NaN,instant_payment,cancelled,NaN,5.0,Instant Payment Cash Request 11444,2020-08-08 06:33:06.244651+00,2020-11-04 16:01:08.332978+00,NaN,NaN,NaN,after
4605,5185,NaN,instant_payment,cancelled,NaN,5.0,Instant Payment Cash Request 11788,2020-08-26 09:39:37.362933+00,2020-11-04 16:01:36.492576+00,NaN,NaN,NaN,after
11870,3590,NaN,instant_payment,cancelled,NaN,5.0,Instant Payment Cash Request 12212,2020-08-12 14:20:06.657075+00,2020-11-04 16:01:53.106416+00,NaN,NaN,NaN,after


In [15]:
# get fees with total_amount different than 5

fees[fees["total_amount"] != 5]

,id,cash_request_id,type,status,category,total_amount,reason,created_at,updated_at,paid_at,from_date,to_date,charge_moment
20604,15552,22799.0,instant_payment,accepted,NaN,10.0,Instant Payment Cash Request 22799,2020-10-21 13:01:52.493241+00,2021-01-21 15:42:51.372269+00,2020-10-30 00:18:41+00,NaN,NaN,after


In [19]:
# check missing data

fees.isna().sum()

id                     0
cash_request_id        4
type                   0
status                 0
category           18865
total_amount           0
reason                 0
created_at             0
updated_at             0
paid_at             5530
from_date          13295
to_date            13295
charge_moment          0
dtype: int64

In [ ]:
# check data types

fees.dtypes

id                   int64
cash_request_id    float64
type                object
status              object
category            object
total_amount       float64
reason              object
created_at          object
updated_at          object
paid_at             object
from_date           object
to_date             object
charge_moment       object
dtype: object

In [31]:
# check the format of the columns with dates


display(fees["created_at"].apply(type).unique())    # str
display(fees["updated_at"].apply(type).unique())    # str
display(fees["paid_at"].apply(type).unique())       # str, float
display(fees["from_date"].apply(type).unique())     # str, float
display(fees["to_date"].apply(type).unique())       # str, float



array([<class 'str'>], dtype=object)

array([<class 'str'>], dtype=object)

array([<class 'str'>, <class 'float'>], dtype=object)

array([<class 'float'>, <class 'str'>], dtype=object)

array([<class 'float'>, <class 'str'>], dtype=object)

In [40]:
# some columns with dates have entries with the data type float.
# check if they're the same entries where data is not available NaN (in case they're not, it can indicate corrupted data)

fees_with_valid_dates = fees[["paid_at", "from_date", "to_date"]].apply(pd.to_datetime, errors="coerce")

missing_count = fees.shape[0] - fees_with_valid_dates.shape[0]

display(missing_count) # 0




0